# ECON 0150 | Replication Notebook

**Title:** Gas Prices and Consumer Sentiment

**Original Authors:** Nathan; Schiller

**Original Date:** Fall 2024

---

This notebook replicates the analysis from a student final project in ECON 0150: Economic Data Analysis.

## About This Replication

**Research Question:** Is an increase in gas prices associated with a subsequent decrease in consumer sentiment?

**Data Source:** FRED - Gas prices (GASREGW), Consumer Sentiment (UMCSENT), Food CPI (CPIUFDSL) - monthly data 2000-present

**Methods:** OLS regression with lagged dependent variable: next month's sentiment change ~ this month's gas price change

**Main Finding:** Negative relationship but not statistically significant (coef = -2.53, p = 0.080).

**Course Concepts Used:**
- Time series analysis
- First differences
- Lagged variables
- Multiple regression with controls

---
## Step 0 | Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

In [ ]:
# Load data from course website
base_url = 'https://tayweid.github.io/econ-0150/projects/replications/0044/data/'

# Load all three datasets
gas = pd.read_csv(base_url + 'GASREGW.csv')
sentiment = pd.read_csv(base_url + 'UMCSENT.csv')
food = pd.read_csv(base_url + 'CPIUFDSL.csv')

print(f"Gas prices: {len(gas)} observations")
print(f"Consumer sentiment: {len(sentiment)} observations")
print(f"Food CPI: {len(food)} observations")

---
## Step 1 | Data Preparation

In [ ]:
# Merge datasets
data = pd.merge(gas, sentiment, on='observation_date')
data = pd.merge(data, food, on='observation_date')

# Calculate changes
data['change_gas'] = data['GASREGW'].diff()  # This month's gas change
data['following_month_change_sentiment'] = data['UMCSENT'].diff().shift(-1)  # Next month's sentiment change
data['change_food'] = data['CPIUFDSL'].diff()  # This month's food price change (control)

# Drop NA rows
data = data.dropna()

print(f"Merged data: {len(data)} observations")
data.head()

---
## Step 2 | Data Exploration

In [ ]:
# Summary statistics
print("Summary Statistics:")
print(data[['change_gas', 'following_month_change_sentiment', 'change_food']].describe())

In [ ]:
# Correlation
correlation = data['change_gas'].corr(data['following_month_change_sentiment'])
print(f"Correlation between gas price change and next month sentiment change: {correlation:.3f}")

---
## Step 3 | Visualization

In [ ]:
# Scatter plot: Gas change vs Next month sentiment change
plt.figure(figsize=(10, 6))
sns.scatterplot(x='change_gas', y='following_month_change_sentiment', data=data, alpha=0.6)
sns.regplot(x='change_gas', y='following_month_change_sentiment', data=data, 
            scatter=False, color='red', ci=None)
plt.xlabel('Monthly Change in Gas Price ($)')
plt.ylabel('Change in Consumer Sentiment (Following Month)')
plt.title('Gas Price Changes vs Following Month Sentiment Changes')
plt.grid(True, alpha=0.3)
plt.show()

---
## Step 4 | Statistical Analysis

In [ ]:
# Model 1: Simple regression
model_1 = smf.ols('following_month_change_sentiment ~ change_gas', data=data).fit()
print("Model 1: Sentiment Change ~ Gas Change")
print(model_1.summary().tables[1])

In [ ]:
# Model 2: With food price control
model_2 = smf.ols('following_month_change_sentiment ~ change_gas + change_food', data=data).fit()
print("\nModel 2: With Food Price Control")
print(model_2.summary().tables[1])

In [ ]:
# Residual plot
residuals = model_1.resid
fitted = model_1.fittedvalues

plt.figure(figsize=(10, 5))
plt.scatter(fitted, residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Key results
print("\n" + "="*50)
print("KEY RESULTS")
print("="*50)
print(f"\nNull Hypothesis: Gas price changes don't affect future sentiment (beta = 0)")
print(f"\nModel 1 (Simple):")
print(f"  Gas change coefficient: {model_1.params['change_gas']:.4f}")
print(f"  P-value: {model_1.pvalues['change_gas']:.3f}")
print(f"  Significant at 0.05? {'Yes' if model_1.pvalues['change_gas'] < 0.05 else 'No'}")
print(f"\nModel 2 (With Food Control):")
print(f"  Gas change coefficient: {model_2.params['change_gas']:.4f}")
print(f"  P-value: {model_2.pvalues['change_gas']:.3f}")

---
## Step 5 | Results Interpretation

### Key Findings

| Model | Gas Coefficient | P-value | Significant? |
|-------|----------------|---------|-------------|
| Simple | -2.53 | 0.080 | No (at 0.05) |
| With food control | -2.55 | 0.077 | No |

1. **Negative Direction:** Higher gas prices are associated with decreased sentiment (as expected)

2. **Not Statistically Significant:** p = 0.08 is above the 0.05 threshold

3. **Food Control Doesn't Help:** Adding food prices as a control doesn't change the result

### Economic Interpretation

Why might gas prices affect sentiment?
- **Visible prices:** Gas prices are highly salient (posted on signs)
- **Frequent purchases:** Consumers buy gas regularly
- **Budget impact:** Affects transportation costs for many households

### Why Not Significant?

- **Small effect size:** The true effect may be modest
- **Noise:** Sentiment is affected by many factors
- **Sample size:** More data might find significance
- **Timing:** One-month lag may not capture the full effect

---
## Replication Exercises

### Exercise 1: Different Lags
Try 2-month or 3-month lags. Does the relationship strengthen?

### Exercise 2: Levels vs Changes
Regress sentiment levels on gas price levels. How do results differ?

### Exercise 3: Asymmetry
Do gas price increases affect sentiment differently than decreases?

### Challenge Exercise
Research the behavioral economics of gas prices. Why are they psychologically salient?

In [ ]:
# Your code for exercises

# Example: 2-month lag
# data['sentiment_2mo_lag'] = data['UMCSENT'].diff().shift(-2)
# model_2mo = smf.ols('sentiment_2mo_lag ~ change_gas', data=data.dropna()).fit()
# print(model_2mo.summary().tables[1])